In [21]:
import json
import itertools
import math

In [22]:
data = json.load(open("data/data.json"))

In [23]:
def data_grab(station, time):
    try:
        return data["stations"][station]["times"][time]["all_fare_classes"]["ridership"]
    except KeyError:
        return 0

favorabilities = {}

def favorability(s1, s2):
    if (s1, s2) in favorabilities:
        return favorabilities[(s1, s2)]
    else:
        # print(f"falling favorability(s1: {s1}, s2: {s2})")
        out = 0
        for time in range(24):
            time = f"{time:02}:00:00.000"
            out += min(data_grab(s1, time), data_grab(s2, time))
        try:
            out = out/math.dist([float(data["stations"][s1]["longitude"]), float(data["stations"][s1]["longitude"])], [float(data["stations"][s2]["longitude"]), float(data["stations"][s2]["longitude"])])
        except ZeroDivisionError:
            out = out/(10**20)
        favorabilities[(s1, s2)] = out
        return out

In [24]:
stations = set(data["stations"].keys())

n = 5 #number of lines

In [25]:
combs = itertools.combinations(stations, n)


leading = [("foo", "bar", "egg", "spam", "hello"), 1_000_000_000_000_000]
counter = 0

print((combs))

In [26]:
for comb in combs:
    counter+=1
    if counter%1_000_000 == 0: print(counter, leading)
    val = 0
    for i in range(n):
        for j in range(i+1, n):
            val += favorability(comb[i], comb[j])
    if val < leading[1]:
        leading = [comb, val]

print(leading)

1000000 [('46 St (M,R)', 'Prospect Av (2,5)', 'Far Rockaway-Mott Av (A)', 'Beach 105 St (A,S)', 'Tompkinsville (SIR)'), 299233.4048860624]
2000000 [('46 St (M,R)', 'Prospect Av (2,5)', 'Beach 90 St (A,S)', 'Beach 105 St (A,S)', 'Tompkinsville (SIR)'), 285466.5132992024]


KeyboardInterrupt: 

In [27]:
# Run the above code and save the final separated stations here

#station_origins = ['Beach 105 St (A,S)', '85 St-Forest Pkwy (J)', 'Tremont Av (B,D)', '233 St (2,5)', 'Canal St (1)', 'Rector St (1)', 'Pelham Pkwy (5)', 'Lorimer St (L)/Metropolitan Av (G)', 'Aqueduct-N Conduit Av (A)', 'Broad Channel (A,S)', 'Beach 98 St (A,S)']

#station_origins = ['Avenue J (Q)', '21 St (G)', 'Beach 98 St (A,S)', 'Broad Channel (A,S)', 'Beach 105 St (A,S)']

station_origins = ['46 St (M,R)', 'Prospect Av (2,5)', 'Beach 90 St (A,S)', 'Beach 105 St (A,S)', 'Tompkinsville (SIR)']

lines = [set([k]) for k in station_origins]
print(lines)

[{'46 St (M,R)'}, {'Prospect Av (2,5)'}, {'Beach 90 St (A,S)'}, {'Beach 105 St (A,S)'}, {'Tompkinsville (SIR)'}]


In [28]:
# included = set()

# while (stations - included) != set():
#     print(len(included))
#     for line in lines:
#         leading = leading = ("fakestation", -1)
#         for station in stations:
#             if station not in line:
#                 val = 0
#                 for elem in line:
#                     val += favorability(station, elem)
#                 if val > leading[1]: leading = [station, val]
#         line.add(leading[0])
#         included.add(leading[0])

for station in stations:
    leading = (-1, -1)
    for line_ind in range(len(lines)):
        line = lines[line_ind]
        if len(line) < 85:
            utility = 0
            for elem in line:
                utility += favorability(station, elem)
            utility /= len(line)
            if utility > leading[1]: leading = (line_ind, utility)
    lines[leading[0]].add(station)

print(lines)





[{'46 St (M,R)', 'Prospect Av (2,5)', '34 St-Penn Station (1,2,3)', 'Sterling St (2,5)', 'Clinton-Washington Avs (C)', '45 St (R)', '135 St (2,3)', 'Franklin Av (2,3,4,5)/Botanic Garden (S)', '125 St (A,C,B,D)', 'Nevins St (2,3,4,5)', 'Winthrop St (2,5)', 'Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)', 'Court Sq (E,G,M,7)', '28 St (6)', '47-50 Sts-Rockefeller Ctr (B,D,F,M)', '36 Av (N,W)', '163 St-Amsterdam Av (C)', 'Newkirk Plaza (B,Q)', 'Avenue U (F)', 'Classon Av (G)', '71 St (D)', 'Church Av (B,Q)', 'Bay Pkwy (F)', 'Fort Hamilton Pkwy (F,G)', 'Beverley Rd (Q)', '34 St-Herald Sq (B,D,F,M,N,Q,R,W)', '175 St (A)', '33 St (6)', 'Broad St (J,Z)', 'Eastern Pkwy-Brooklyn Museum (2,3)', 'Bedford Av (L)', 'Crown Hts-Utica Av (3,4)', '59 St-Columbus Circle (A,B,C,D,1)', 'Avenue H (Q)', 'Bay Ridge-95 St (R)', 'Parkside Av (Q)', '68 St-Hunter College (6)', 'Cortelyou Rd (Q)', 'Lafayette Av (C)', 'Grand St (B,D)', '155 St (B,D)', 'Broadway-Lafayette St (B,D,F,M)/Bleecker St (6)', 'Grand Army Pl

In [29]:
for line in lines:
    print(f"testing {len(line)}")

testing 85
testing 85
testing 85
testing 85
testing 92


In [30]:
lines = [list(line) for line in lines]
data["other_data"]["new_lines"] = lines

In [31]:
out_file = open("data/data.json", "w") 
  
json.dump(data, out_file) 
  
out_file.close()